In [1]:
from numba import njit
import numpy as np


# Below is the code for the general model (Model 1). To create the other models, simply set the appropriate model parameters to 0. 

# Example: Model 2 only uses error correction terms correcting towards the intended movement aim. Thus, for Model 2,  $$\beta^{target, x} = \beta^{target, y} = 0$$

In [2]:
@njit
def Model_1(error_condition = 1, reward_condition = 1, sigma_m = np.array([.038, .06]),
            sigma_e = np.array([.051, .09]), sigma_p = np.array([.01, .01]),
            alpha = .4,
            beta_aim_relevant = .4, beta_target_relevant = .4,
            beta_aim_irrelevant = .4, beta_target_irrelevant = .4,
            right_bound = .3, left_bound = -.3):
     
    aim_X, aim_Y = 0, 0
    
    upp_y = .06
    low_y = -.06 
    
    recorded_attempt = np.empty((200, 2))
    history = np.zeros(200)
    
    previous_outcome = 0
    for i in range(200):
        expl_x, expl_y = (1 - previous_outcome)*np.random.normal(0, sigma_e[0]), (1 - previous_outcome)*np.random.normal(0, sigma_e[1])
        motor_x, motor_y = np.random.normal(0, sigma_m[0]), np.random.normal(0, sigma_m[1])
        
        reach_x = aim_X + motor_x + expl_x
        reach_y = aim_Y + motor_y + expl_y 
        
        recorded_attempt[i] = reach_x, reach_y
        
        if (((reach_x <= right_bound) and (reach_x >= left_bound)) and ((reach_y <= upp_y) and (reach_y >= low_y))):
            aim_X  = aim_X + reward_condition * alpha*expl_x - error_condition*(beta_target_relevant   * (reach_x - 0) + beta_aim_relevant   * (reach_x - aim_X))
            aim_Y  = aim_Y + reward_condition * alpha*expl_y - error_condition*(beta_target_irrelevant * (reach_y - 0) + beta_aim_irrelevant * (reach_y - aim_Y))

            previous_outcome = 1
            history[i] = 1
        else:
            aim_X  = aim_X  - error_condition*(beta_target_relevant   * (reach_x - 0) + beta_aim_relevant   * (reach_x - aim_X)) 
            aim_Y  = aim_Y  - error_condition*(beta_target_irrelevant * (reach_y - 0) + beta_aim_irrelevant * (reach_y - aim_Y))  
            
            previous_outcome = 0
            
    
    return recorded_attempt, history

compile_test=Model_1()
